In [31]:
import importlib
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from datetime import datetime
from datetime import date
from datetime import timedelta

In [32]:
import PrjtCF_module as cf

# Outline
* 주거시설 등 개발과 동시에 분양을 하는 사업의 재무모델
* PF대출을 통해 토지비 및 공사비 등 개발사업 자금을 조달
* 금융비용은 PF대출금으로 충당
* 분양대금의 일부는 사업비로, 일부는 대출금 상환재원으로 사용
* PF대출금은 필요에 따라 한도대로 인출

# Input Index Data

In [54]:
# 기간 Index 설정
# prjt index : 사업기간 전 기간에 대한 index로 모델의 base가 되는 index
# cstrn index : 공사기간에 대한 index로 착공일로부터 준공일까지의 기간에 대한 index
# loan index : 대출기간에 대한 index로 대출 실행일로부터 대출 상환일 까지의 기간에 대한 index
# sales index : 분양기간에 대한 index로 분양개시일로부터 분양 종료일 까지의 기간 index
idx = cf.PrjtIndex(idxname=['prjt', 'cstrn', 'loan', 'sales'],
                   start=['2021-08', '2021-10', '2021-10', '2021-12'],
                   periods=[24+1, 18+1, 20+1, 16+1],
                   freq='M')

# 공정률 Series 설정 : cstrn index 기간 중 적용되는 기간별 공정률 설정
idx.prcs = Series(np.ones(len(idx.cstrn)) / len(idx.cstrn),
                  index=idx.cstrn.index)

# Input Financing Condition Data

In [55]:
equity = cf.Loan(idx, idx, amt_ntnl=10_000)

tra = cf.Loan(idx, idx.loan, amt_ntnl=100_000,
              rate_fee = 0.01,
              rate_IR = 0.10)

# Input Sales Data

In [56]:
dct_sales = {}

#### 분양상품A ####
# 분양매출 및 납입 일정
slsA = cf.Account(idx)
slsA.sls_amt = 150_000
slsA.csh_idx = idx.sales[[0, 3, 6, 9, 12, 16]]
slsA.csh_rate = Series([0.1, 0.1, 0.1, 0.1, 0.1, 0.5], 
                       index=slsA.csh_idx)
# 분양매출 가정
slsA.sls_rate = Series(np.array([0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1]),
                       index=idx.sales[[0, 2, 4, 6, 7, 8, 9, 12, 14, 16]])
slsA.sls_plan = slsA.sls_amt * slsA.sls_rate

# Input Cost Data

In [57]:
dct_cost = {}

# 토지비: 최초 1회 지급
lnd = cf.Account(idx)
lnd.addscdd(idx.cstrn[0], 30_000)
dct_cost['lnd'] = lnd

# 공사비: 공정률에 따라 지급
cstrn = cf.Account(idx)
cstrn.addscdd(idx.cstrn.index, 50_000 * idx.prcs)
dct_cost['cstrn'] = cstrn

cost = cf.Merge(dct_cost)
cost.lnd = cost.dct['lnd']
cost.cstrn = cost.dct['cstrn']

# Execution Cash Flow

### 1) 사전 설정

In [58]:
# Make accounts
oprtg = cf.Account(idx) # 운영현금흐름의 입출을 위한 운영계좌
sales = cf.Account(idx) # 분양수입금의 입출을 위한 분양수입금 계좌
rpyacc = cf.Account(idx) # 상환자금 관리를 위한 대출금 상환 계좌

In [59]:
# Calculate cash amount required and withdraw loan.
class wtdrw_mngmnt:
    def __init__(self, idxno, cstmng):
        self.idxno = idxno
        self.cstmng = cstmng
        
        self.amt_wtdrw = 0 # 인출된 금액
        self.rsdl_wtdrw = self.wtdrw_exptd # 인출필요금액(비용 등)
        
    @property
    def wtdrw_exptd(self):
        """총 지출필요금액을 확인한 후 운영계좌 잔액을 초과하는 금액
        (추가 인출이 필요한 금액)을 계산하여 반환"""
        cst_exptd = self.cstmng.ttl_exptd # 총 지출필요금액
        oprtg_bal = oprtg.bal_end[self.idxno] # 운영계좌 잔액
        
        amt_rqrd = max(cst_exptd - oprtg_bal, 0)
        # 지출필요금액에 대하여 운영계좌 잔액을 초과하는 금액 계산

        return amt_rqrd
    
    def wtdrw_equity(self, eqty):
        """equity instance에 대하여 idxno에 대응하는 인출예정금액(sub_scdd)을
        조회하여 운영계좌로 이체"""
        if eqty.is_wtdrbl:
            amt_wtdrw = eqty.ntnl.sub_scdd[self.idxno]
            eqty.ntnl.send(self.idxno, amt_wtdrw, oprtg)
            
    def wtdrw_loan(self, loan):
        """loan instance에 대하여 idxno에 대응하는 누적인출가능잔액 확인,
        누적인출가능잔액 내에서 인출필요금액(비용 등)을 운영계좌로 이체"""
        if loan.is_wtdrbl:
            ntnl_sub_rsdl = loan.ntnl.sub_rsdl_cum[self.idxno] # 누적인출가능잔액
            tmp_wtdrw = min(ntnl_sub_rsdl, self.rsdl_wtdrw)
            # 누적인출가능잔액과 인출필요금액을 비교하여 적은 금액을 대입
            
            loan.ntnl.send(self.idxno, tmp_wtdrw, oprtg)
            # 추가 인출필요금액을 운영계좌로 이체(누적인출가능잔액 내에서)
            
            self.amt_wtdrw += tmp_wtdrw # 인출된 금액
            self.rsdl_wtdrw -= tmp_wtdrw # 인출 후 잔여 인출필요금액
            

In [60]:
######## 나중에 개별 sales instance를 입력받아서 처리하는 것으로 수정 요망 ######


# receive sales amount
class sls_mngmnt:
    def __init__(self, idxno, sales):
        self.idxno = idxno
        self.sls = sales
        
        # 해당 인덱스 기간 중 인출 예정 sales amount
        self.sls_amt = 0
        
    # Check sales plan and input sales data
    def make_sls_plan(self):
        try:
            # check sales plan
            sls_amt = self.sls.sls_plan.loc[self.idxno]
            try:
                # input sales amount on this index no.
                self.sls.addamt(self.idxno, sls_amt)
                # input sales cash schedule on sales cash index.
                self.sls.subscdd(self.sls.csh_idx, sls_amt * self.sls.csh_rate)
            except AttributeError as err:
                print("AttributeError", err)
        except:
            pass
        
        self.sls_amt = self.sls.sub_rsdl_cum[self.idxno]
        
    # Receive sales amount on sales account
    def rcv_slsamt(self, acnt_sales):
        self.sls.send(self.idxno, self.sls_amt, acnt_sales)
        
    # Transfer amount to operating account
    def trsf_oprtg(self, acnt_oprtg):
        sales.send(self.idxno, self.sls_amt, acnt_oprtg)


In [61]:
# Calculate expected cost amount and pay cost.
class cst_mngmnt:
    def __init__(self, idxno):
        self.idxno = idxno
        
        # 해당 인덱스 기간 중 cost계좌 상 지출이 예정되어 있는 금액
        self.lnd = cost['lnd'].add_scdd[self.idxno]
        self.cstrn = cost['cstrn'].add_scdd[self.idxno]
        
        # cost 계좌 상 예정된 지출금의 합계액
        self.cstamt = cost.add_scdd[self.idxno] 
        
        # 해당 인덱스 기간 중 loan 계좌 상 지출 필요 금액
        self.trafee = tra.fee.add_scdd[self.idxno]
        self.traIR = -tra.ntnl.bal_strt[self.idxno] * tra.IR.rate
        
    @property
    def ttl_exptd(self):
        """전체 운영비용(cost) 및 금융비용(fee, IR)의 합을 반환"""
        self.ttlsum = self.cstamt
        self.ttlsum = (self.ttlsum +
                       self.trafee + 
                       self.traIR)
        return self.ttlsum
    
    def pay_oprtcst(self):
        """운영계좌에서 운영비용(cost) 지출"""
        # 운영계좌에서 토지비 지출
        oprtg.send(self.idxno, self.lnd, cost['lnd'])
        
        # 운영계좌에서 공사비 지출
        oprtg.send(self.idxno, self.cstrn, cost['cstrn'])
        
    def pay_fnclcst(self):
        """운영계좌에서 금융비용 지출"""
        # 운영계좌에서 대출금 수수료 지급
        oprtg.send(self.idxno, self.trafee, tra.fee)
        
        # 운영계좌에서 대출금 이자 지급
        oprtg.send(self.idxno, self.traIR, tra.IR)
        

In [62]:
# Calculate expected repayment of loan and repay loan.
class repay_mngmnt:
    """입력된 Loan instance에 대해서 
    - 상환요구금액 계산
    - 상환계좌로 자금 이체
    - 상환계좌에서 loan 계좌로 상환 처리"""
    def __init__(self, idxno, loan):
        self.idxno = idxno
        self.loan = loan
        
        # 상환요구금액 계산
        self.exptd_rpy_cum = self.loan.ntnl.add_rsdl_cum[self.idxno]
        self.ntnl_bal_end = -self.loan.ntnl.bal_end[self.idxno]
        self.rpy_amt = min(self.exptd_rpy_cum, self.ntnl_bal_end)
        
    # Transfer repayment amount to repayment account
    def trsf_rpy(self):
        oprtg.send(self.idxno, self.rpy_amt, rpyacc)
    
    # Repay loan from repayment account
    def rpy_ntnl(self):
        rpyacc.send(self.idxno, self.rpy_amt, self.loan.ntnl)
        

### 2) Cash Flow 실행

In [63]:
# Excute cash flow
for idxno in idx.index:
    # If it's initial date then set loan withdrawble.
    equity.set_wtdrbl_intldate(idxno)
    tra.set_wtdrbl_intldate(idxno)
        
    # Sales : 분양수입대금 sales schedule에 따라 운영계좌로 이체
    sls = sls_mngmnt(idxno, slsA)
    sls.make_sls_plan()
    sls.rcv_slsamt(sales)
    sls.trsf_oprtg(oprtg)
    
    # cost instance 생성, 필요지출금액을 인자로 받아, 조달금액 계산 후 운영계좌 입금
    cst = cst_mngmnt(idxno)
    wtdrw = wtdrw_mngmnt(idxno, cst)
    wtdrw.wtdrw_equity(equity)
    wtdrw.wtdrw_loan(tra)
    
    # 운영계좌에서 토지비, 공사비 등 각종 비용 지출
    cst.pay_oprtcst()
    
    # 운영계좌에서 각종 금융비용 지출
    cst.pay_fnclcst()
    
    # 대출금 상환 프로세스 진행
    repay = repay_mngmnt(idxno, tra) # Loan instance class 생성
    repay.trsf_rpy() # 상환 예정 금액을 상환계좌로 이체
    repay.rpy_ntnl() # 상환계좌에서 순서에 따라 상환금 인출
    
    # If it was maturity date then set back loan unwithdrawble.
    equity.setback_wtdrbl_mtrt(idxno)
    tra.setback_wtdrbl_mtrt(idxno)

In [64]:
# Merge Accounts
dct_acc = {'equity':equity,
           'tra_ntnl':tra.ntnl,
           'tra_fee':tra.fee,
           'tra_IR':tra.IR,
           'slsA':slsA,
           'cost_lnd':cost['lnd'],
           'cost_cstrn':cost['cstrn'],
           'oprtg':oprtg,
           'rpyacc':rpyacc}

# Print Result

In [65]:
# 현금흐름 종합
rslt_df = DataFrame({('equity.ntnl', 'amt_sub'):equity.ntnl.amt_sub[:],
                     ('slsA', 'amt_add'):slsA.amt_add[:],
                     ('slsA', 'amt_sub'):slsA.amt_sub[:],
                     ('tra.ntnl', 'amt_sub'):tra.ntnl.amt_sub[:],
                     ('tra.ntnl', 'amt_add'):tra.ntnl.amt_add[:],
                     ('tra.ntnl', 'bal_end'):tra.ntnl.bal_end[:],
                     ('tra.fee', 'amt_add'):tra.fee.amt_add[:],
                     ('tra.IR', 'amt_add'):tra.IR.amt_add[:],
                     ('cost_lnd', 'amt_add'):cost['lnd'].amt_add[:],
                     ('cost_cstrn', 'amt_add'):cost['cstrn'].amt_add[:],
                     ('oprtg', 'amt_add'):oprtg.amt_add[:],
                     ('oprtg', 'amt_sub'):oprtg.amt_sub[:],
                     ('oprtg', 'bal_end'):oprtg.bal_end[:],
                     ('rpyacc', 'amt_add'):rpyacc.amt_add[:],
                     ('rpyacc', 'amt_sub'):rpyacc.amt_sub[:],
                     ('rpyacc', 'bal_end'):rpyacc.bal_end[:]})
rslt_df.fillna(0).applymap(lambda x: f"{x:,.0f}")

equity.ntnl    slsA         tra.ntnl                  tra.fee  \
               amt_sub amt_add amt_sub  amt_sub amt_add  bal_end amt_add   
2021-08-31      10,000       0       0        0       0        0       0   
2021-09-30           0       0       0        0       0        0       0   
2021-10-31           0       0       0   23,632       0  -23,632   1,000   
2021-11-30           0       0       0    2,829       0  -26,460       0   
2021-12-31           0  15,000   1,500    1,352       0  -27,812       0   
2022-01-31           0       0       0    2,863       0  -30,676       0   
2022-02-28           0  15,000   1,500    1,387       0  -32,063       0   
2022-03-31           0       0   3,000        0       0  -32,063       0   
2022-04-30           0  15,000   3,000        0       0  -32,063       0   
2022-05-31           0       0       0    2,696       0  -34,759       0   
2022-06-30           0  15,000   9,000        0       0  -34,759       0   
2022-07-31           0  15,000   4,500        0       0  -34,759       0   
2022-08-31           0  15,000   4,500        0       0  -34,759       0   
2022-09-30           0  15,000  15,000        0       0  -34,759       0   
2022-10-31           0       0       0        0       0  -34,759       0   
2022-11-30           0       0       0        0       0  -34,759       0   
2022-12-31           0  15,000  18,000        0       0  -34,759       0   
2023-01-31           0       0       0        0       0  -34,759       0   
2023-02-28           0  15,000   7,500        0       0  -34,759       0   
2023-03-31           0       0       0        0       0  -34,759       0   
2023-04-30           0  15,000  82,500        0       0  -34,759       0   
2023-05-31           0       0       0        0       0  -34,759       0   
2023-06-30           0       0       0        0  34,759        0       0   
2023-07-31           0       0       0        0       0        0       0   
2023-08-31           0       0       0        0       0        0       0   

            tra.IR cost_lnd cost_cstrn   oprtg                   rpyacc  \
           amt_add  amt_add    amt_add amt_add amt_sub  bal_end amt_add   
2021-08-31       0        0          0  10,000       0   10,000       0   
2021-09-30       0        0          0       0       0   10,000       0   
2021-10-31       0   30,000      2,632  23,632  33,632        0       0   
2021-11-30     197        0      2,632   2,829   2,829        0       0   
2021-12-31     221        0      2,632   2,852   2,852        0       0   
2022-01-31     232        0      2,632   2,863   2,863        0       0   
2022-02-28     256        0      2,632   2,887   2,887        0       0   
2022-03-31     267        0      2,632   3,000   2,899      101       0   
2022-04-30     267        0      2,632   3,000   2,899      202       0   
2022-05-31     267        0      2,632   2,696   2,899        0       0   
2022-06-30     290        0      2,632   9,000   2,921    6,079       0   
2022-07-31     290        0      2,632   4,500   2,921    7,658       0   
2022-08-31     290        0      2,632   4,500   2,921    9,236       0   
2022-09-30     290        0      2,632  15,000   2,921   21,315       0   
2022-10-31     290        0      2,632       0   2,921   18,394       0   
2022-11-30     290        0      2,632       0   2,921   15,473       0   
2022-12-31     290        0      2,632  18,000   2,921   30,551       0   
2023-01-31     290        0      2,632       0   2,921   27,630       0   
2023-02-28     290        0      2,632   7,500   2,921   32,209       0   
2023-03-31     290        0      2,632       0   2,921   29,288       0   
2023-04-30     290        0      2,632  82,500   2,921  108,866       0   
2023-05-31     290        0          0       0     290  108,577       0   
2023-06-30     290        0          0       0  35,049   73,528  34,759   
2023-07-31       0        0          0       0       0   73,528       0   
2023-08-

In [66]:
# 현금흐름 요약
rslt_df = DataFrame({('equity.ntnl', 'amt_sub'):equity.ntnl.amt_sub[:],
                     ('slsA', 'amt_add'):slsA.amt_add[:],
                     ('slsA', 'amt_sub'):slsA.amt_sub[:],
                     ('tra.ntnl', 'amt_sub'):tra.ntnl.amt_sub[:],
                     ('tra.ntnl', 'amt_add'):tra.ntnl.amt_add[:],
                     ('oprtg', 'amt_add'):oprtg.amt_add[:],
                     ('tra.fee', 'amt_add'):tra.fee.amt_add[:],
                     ('tra.IR', 'amt_add'):tra.IR.amt_add[:],
                     ('cost_lnd', 'amt_add'):cost['lnd'].amt_add[:],
                     ('cost_cstrn', 'amt_add'):cost['cstrn'].amt_add[:],
                     ('oprtg', 'amt_sub'):oprtg.amt_sub[:],
                     ('oprtg', 'bal_end'):oprtg.bal_end[:]})
rslt_df.fillna(0).applymap(lambda x: f"{x:,.0f}")

equity.ntnl    slsA         tra.ntnl           oprtg tra.fee  \
               amt_sub amt_add amt_sub  amt_sub amt_add amt_add amt_add   
2021-08-31      10,000       0       0        0       0  10,000       0   
2021-09-30           0       0       0        0       0       0       0   
2021-10-31           0       0       0   23,632       0  23,632   1,000   
2021-11-30           0       0       0    2,829       0   2,829       0   
2021-12-31           0  15,000   1,500    1,352       0   2,852       0   
2022-01-31           0       0       0    2,863       0   2,863       0   
2022-02-28           0  15,000   1,500    1,387       0   2,887       0   
2022-03-31           0       0   3,000        0       0   3,000       0   
2022-04-30           0  15,000   3,000        0       0   3,000       0   
2022-05-31           0       0       0    2,696       0   2,696       0   
2022-06-30           0  15,000   9,000        0       0   9,000       0   
2022-07-31           0  15,000   4,500        0       0   4,500       0   
2022-08-31           0  15,000   4,500        0       0   4,500       0   
2022-09-30           0  15,000  15,000        0       0  15,000       0   
2022-10-31           0       0       0        0       0       0       0   
2022-11-30           0       0       0        0       0       0       0   
2022-12-31           0  15,000  18,000        0       0  18,000       0   
2023-01-31           0       0       0        0       0       0       0   
2023-02-28           0  15,000   7,500        0       0   7,500       0   
2023-03-31           0       0       0        0       0       0       0   
2023-04-30           0  15,000  82,500        0       0  82,500       0   
2023-05-31           0       0       0        0       0       0       0   
2023-06-30           0       0       0        0  34,759       0       0   
2023-07-31           0       0       0        0       0       0       0   
2023-08-31           0       0       0        0       0       0       0   

            tra.IR cost_lnd cost_cstrn   oprtg           
           amt_add  amt_add    amt_add amt_sub  bal_end  
2021-08-31       0        0          0       0   10,000  
2021-09-30       0        0          0       0   10,000  
2021-10-31       0   30,000      2,632  33,632        0  
2021-11-30     197        0      2,632   2,829        0  
2021-12-31     221        0      2,632   2,852        0  
2022-01-31     232        0      2,632   2,863        0  
2022-02-28     256        0      2,632   2,887        0  
2022-03-31     267        0      2,632   2,899      101  
2022-04-30     267        0      2,632   2,899      202  
2022-05-31     267        0      2,632   2,899        0  
2022-06-30     290        0      2,632   2,921    6,079  
2022-07-31     290        0      2,632   2,921    7,658  
2022-08-31     290        0      2,632   2,921    9,236  
2022-09-30     290        0      2,632   2,921   21,315  
2022-10-31     290        0      2,632   2,921   18,394  
2022-11-30     290        0      2,632   2,921   15,473  
2022-12-31     290        0      2,632   2,921   30,551  
2023-01-31     290        0      2,632   2,921   27,630  
2023-02-28     290        0      2,632   2,921   32,209  
2023-03-31     290        0      2,632   2,921   29,288  
2023-04-30     290        0      2,632   2,921  108,866  
2023-05-31     290        0          0     290  108,577  
2023-06-30     290        0          0  35,049   73,528  
2023-07-31       0        0          0       0   73,528  
2023-08-31       0        0          0       0   73,528